In [ ]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Embedding

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Reading the Trojan Dataset
df = pd.read_csv('/content/drive/MyDrive/SIH - Neural Trojan Detectives/Trojan_Detection.csv')

In [ ]:
df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,73217,10.42.0.42-121.14.255.84-49975-80-6,10.42.0.42,49975,121.14.255.84,80,6,17/07/2017 01:18:33,10743584,4,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Trojan
1,72089,172.217.6.226-10.42.0.42-443-49169-17,10.42.0.42,49169,172.217.6.226,443,17,17/07/2017 10:25:25,254217,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Trojan
2,96676,10.42.0.1-10.42.0.42-53-37749-17,10.42.0.42,37749,10.42.0.1,53,17,30/06/2017 07:16:12,1023244,1,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,42891,10.42.0.1-10.42.0.42-53-41352-17,10.42.0.42,41352,10.42.0.1,53,17,13/07/2017 03:48:44,286483,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Trojan
4,169326,10.42.0.151-107.22.241.77-44353-443-6,10.42.0.151,44353,107.22.241.77,443,6,5/7/2017 10:47,65633087,12,...,32,322594.0,0.0,322594,322594,60306983.0,0.0,60306983,60306983,Benign


In [ ]:
df = df.dropna()
df.drop(["Unnamed: 0"], axis = 1).values

df = df.replace("Trojan", 1)
df = df.replace("Benign", 0)

In [ ]:
df.columns

Index(['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Len

In [ ]:
# Modifying the values so that it can fit with the normal data
number = preprocessing.LabelEncoder()

df["Flow ID"] = number.fit_transform(df["Flow ID"])
df[" Source IP"] = number.fit_transform(df[" Source IP"])
df[" Destination IP"] = number.fit_transform(df[" Destination IP"])
df[" Timestamp"] = number.fit_transform(df[" Timestamp"])

df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,73217,46111,7,49975,352,80,6,7173,10743584,4,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
1,72089,74905,7,49169,895,443,17,10145,254217,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
2,96676,9217,7,37749,7,53,17,13354,1023244,1,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,42891,10418,7,41352,7,53,17,789,286483,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,1
4,169326,20763,5,44353,220,443,6,13959,65633087,12,...,32,322594.0,0.0,322594,322594,60306983.0,0.0,60306983,60306983,0


In [ ]:
# Keeping Label column in a different variable
X = df.drop([" Label"], axis = 1).values
y = df[" Label"].values

In [ ]:
# Standardize Features by removing the Mean and Scaling to Unit Variance
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# Data Spliting
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=0, test_size=0.2)

In [ ]:
# Defining the model
model = Sequential()

# CNN Layers
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM Layer
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50, return_sequences=False))

# Fully Connected Layers
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Reshaping X_train and X_test to match the model's input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluating the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Epoch 1/10
1775/1775 [==============================] - 34s 11ms/step - loss: 0.6912 - accuracy: 0.5136 - val_loss: 0.6914 - val_accuracy: 0.5070
Epoch 2/10
1775/1775 [==============================] - 19s 10ms/step - loss: 0.6909 - accuracy: 0.5169 - val_loss: 0.6888 - val_accuracy: 0.5293
Epoch 3/10
1775/1775 [==============================] - 18s 10ms/step - loss: 0.6916 - accuracy: 0.5151 - val_loss: 0.6908 - val_accuracy: 0.5105
Epoch 4/10
1775/1775 [==============================] - 18s 10ms/step - loss: 0.6467 - accuracy: 0.5525 - val_loss: 0.0293 - val_accuracy: 0.9907
Epoch 5/10
1775/1775 [==============================] - 19s 11ms/step - loss: 0.0156 - accuracy: 0.9947 - val_loss: 0.0034 - val_accuracy: 0.9991
Epoch 6/10
1775/1775 [==============================] - 19s 11ms/step - loss: 0.0055 - accuracy: 0.9982 - val_loss: 0.0012 - val_accuracy: 0.9998
Epoch 7/10
1775/1775 [==============================] - 18s 10ms/step - loss: 0.0230 - accuracy: 0.9909 - val_loss: 0.0074 -